# Классификация нового товара по рассчитанным ранее сегментам

# 0. Импорт библиотек

In [115]:
import pandas as pd
import numpy as np
import re
import datetime
import warnings
warnings.filterwarnings('ignore')

# 1. Загрузка исходников

## Приоритеты сайта

In [116]:
# Приоритеты характеристик от Сайта
Prioritet = pd.read_csv('data\site_priorities.zip')

In [117]:
Prioritet.head(2)

,Категория,ID характеристики,Название характеристики,Приоритет,"Частота использования, %",Количество кликов
0,Электронные книги,1000,Объем встроенной памяти,720,0.0013,80
1,Электронные книги,989,Диагональ дисплея,900,0.2591,15525


## Автосегменты из справочника

In [133]:
dim_segment = pd.read_csv('data/dim_auto_segments.zip')
dim_segment.head(2)

,Сегмент,Характеристики,КодыХарактеристик,НомерУровня,КодКатегории,Power,Родитель
0,Тип переплета_пластиковые пружины // Максималь...,"{'Основной цвет_черный', 'Максимальное кол-во ...","{'92316562', '125748', '494916', '126028', '12...",1,AM18123,2,NaN
1,Тип переплета_пластиковые пружины,"{'Ширина_435 мм', 'Максимальное кол-во перфори...","{'63586425', '106142937', '31392352', '125784'...",1,AM18123,1,NaN


In [132]:
dim_segment.describe(include=['object'])

,Сегмент,Характеристики,КодыХарактеристик,КодКатегории,Родитель
count,38509,38509,38509,38509,33817
unique,38401,37215,37237,546,14397
top,Основной цвет_белый,{'Основной цвет_черный'},"{'48365834', '649103', '35644220'}",AM18705,Прочее
freq,8,5,4,577,896


In [120]:
list_auto_segments = dim_segment['КодКатегории'].unique()
len(list_auto_segments)

546

## Иерархия категорий

In [134]:
# Преобразование иерархии в датафрейм
df_hierarchy = pd.read_csv('data/hierarchy.zip')
print(len(df_hierarchy))
df_hierarchy.drop(columns='Unnamed: 0').head(2)

718


,Уровень1,Уровень2,Уровень3,Уровень4,КодКатегории
0,05. DIY и Климат,01. Электроинструмент и Садовая техника,04. Клининговый,Стеклоочиститель портативный,EM85913
1,05. DIY и Климат,01. Электроинструмент и Садовая техника,04. Клининговый,Пылесос строительный,AM24371


## КодТовара - КодХарактеристики

In [122]:
df_dim_product_character = pd.read_csv('data/dim_product_character.zip')
print(len(df_dim_product_character))
df_dim_product_character = df_dim_product_character.drop(columns='Unnamed: 0').astype('str')
df_dim_product_character.head(3)

1949760


,Код,КодХарактеристики,ВидХарактеристики,КодКатегории
0,9973367,101797563,Диапазон входного напряжения сети,AM18166
1,9973367,158143,Комплектация,AM18166
2,9973367,11543,Количество разъемов 15-pin SATA (сервисная),AM18166


In [123]:
df_dim_product_character.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1949760 entries, 0 to 1949759
Data columns (total 4 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   Код                object
 1   КодХарактеристики  object
 2   ВидХарактеристики  object
 3   КодКатегории       object
dtypes: object(4)
memory usage: 59.5+ MB


# 2. Основные настройки и предустановки

## Выбор списка ктегорий для загрузки

In [124]:
list_auto_segments = ['AM18166', 'AM18156', 'AM18165', 'AM18166', 'AM18168', 'AM18171', 'AM18172', 'AM18173', \
    'AM18176', 'AM18178', 'AM18179', 'AM18180', 'AM18181', 'AM18182', 'AM18200'] # Выборка из списка категорий
list_auto_segments

['AM18166',
 'AM18156',
 'AM18165',
 'AM18166',
 'AM18168',
 'AM18171',
 'AM18172',
 'AM18173',
 'AM18176',
 'AM18178',
 'AM18179',
 'AM18180',
 'AM18181',
 'AM18182',
 'AM18200']

## Функции

### Функции преобразования типов данных в таблицах

In [125]:
# определяем функцию для преобразования строки в множество
def string_to_set(string):
    return set(string.split(', '))

# определяем функцию для преобразования множества в список
def set_to_list(set):
    return sorted(list(set))

# определяем функцию для преобразования множества в строку
def set_to_string(set):
    return str(set)

# Преобразовать строку во множество и очистить
def get_string_to_set_v2(string):
    set_1 = set()
    for i in string.split(','):
        i = i.replace("'", "")
        i = i.replace("{", "")
        i = i.replace("}", "")
        i = i.replace(" ", "")
        set_1.add(i)
    return(set_1)

### Функция выделения видов характеристик

In [126]:
def wiew_char_list(x):
    wiew_char_list_x = set(x.split(" // "))
    wiew_char_list_x = [i.split('_')[0] for i in wiew_char_list_x]
    wiew_char_list_x = {code.replace("'", "") for code in wiew_char_list_x}
    return wiew_char_list_x

### Функция создания множества характеристик из массива множеств

In [127]:
def get_set_char(arr):
    set_char = set()
    for i in arr:
        for j in i:
            set_char.add(j)
    return set_char

### Функция для нахождения пересечения множеств в каждой строке

In [128]:
def find_intersection(row):
    return row['КодыХарактеристик'].intersection(row['set_char_for_wiew'])

### Функция группировки и преобразования в наборы

In [129]:

def get_selection(df, lvl, collumn):
    df = df.groupby(lvl)[collumn].apply(', '.join).reset_index()
    df[collumn] = df[collumn].apply(string_to_set)
    return df

# 3. Скрипт алгоритма классификации

In [130]:
counter = 0 # Счетчик
df_tovar_segment_0 = pd.DataFrame()
df_for_dim_segment_0 = pd.DataFrame()
for Category_code in list_auto_segments:
    counter +=1 # Счетчик
    Category_name = df_hierarchy[df_hierarchy['КодКатегории'] == Category_code]['Уровень4'].values[0]
    
    print()
    print('Обработка категории :', Category_code, Category_name)

    # Получение данных по категории
    df_prod_char = df_dim_product_character[df_dim_product_character['КодКатегории'] == Category_code]
    df_0 = df_prod_char.copy() # Копия исодных данных
    
    # Справочник видов и кодов характеристик
    dim_wiew_code = df_prod_char.groupby('ВидХарактеристики')['КодХарактеристики'].apply(', '.join)
    dim_wiew_code = pd.DataFrame(dim_wiew_code).reset_index()
    dim_wiew_code['КодХарактеристики'] = dim_wiew_code['КодХарактеристики'].apply(lambda string: string_to_set(string))
    
    # Задаем номер уровня для расчета
    level_list = [1, 2, 3]
    for level in level_list:
        # Оставляем сегменты нужного уровня из одной категории
        mask_cat = dim_segment['КодКатегории']==Category_code
        dim_segment_filter = dim_segment[mask_cat & (dim_segment['НомерУровня'] == level)]
        
        if len(dim_segment_filter) == 0: # Если сегменты на выбранном уровне отсутствуют - пропускаем этап
            print(f'Уровень {level} отсутствует в справочнике сегментов')
            pass
        else:
            
            ### Найти список уникальных видов характеристик ###
            
            # объединяем все строки в одну с разделителем ", "
            множество_видов_характеристик = ", ".join(dim_segment_filter['Сегмент'])
            # преобразуем строку в множество
            wiew_char_set = set(множество_видов_характеристик.split(" // "))
            wiew_char_set = [i.split('_')[0] for i in wiew_char_set]
            # убираем кавычки из каждого элемента множества
            wiew_char_set = {code.replace("'", "") for code in wiew_char_set}
            # Уникальные виды характеристик
            dim_segment_filter['виды_характеристик'] = dim_segment_filter['Сегмент'].apply(lambda x: wiew_char_list(x))
            
            ### Найти список уникальных кодов характеристик ###
            
            # Уникальные характеристики из справочника. Массив множеств
            arr_set = dim_wiew_code[dim_wiew_code['ВидХарактеристики'].isin(wiew_char_set)]['КодХарактеристики'].values
            # Множество из множеств по сегментам этого уровня
            set_set = get_set_char(arr_set)
            
            ### Обработка справочника сегментов ###
            
            # Преобразование строки во множество
            dim_segment_filter['КодыХарактеристик'] = dim_segment_filter['КодыХарактеристик'].apply(lambda x: get_string_to_set_v2(x))
            dim_segment_filter.head(2)
            
            # Собать множество характеристик только по видам из сегмента
            dim_segment_filter['set_char_for_wiew'] = dim_segment_filter['виды_характеристик'].apply(
                lambda x: get_set_char(dim_wiew_code[dim_wiew_code['ВидХарактеристики'].isin(x)]['КодХарактеристики'].values))
            
            # Оставить только пересеченные множества
            dim_segment_filter['КодыХарактеристик'] = dim_segment_filter.apply(find_intersection, axis=1)
            # Оставить нужные столбцы
            dim_segment_filter = dim_segment_filter[[
                'Сегмент', 'КодыХарактеристик', 'НомерУровня', 'Power', 'Родитель'
                ]].reset_index(drop='index')
            
            ### Обработка категории ###
            
            # Объединить список кодов характеристик товара во множество
            df_0_new = df_prod_char[['Код', 'КодХарактеристики']]
            
            # Условие первичной обработки данных, для первого цикла по категории.
            if level != 1:
                pass
            else:
                # Таблица с наборами в виде множеств
                product_set_code = get_selection(df_0_new, 'Код', 'КодХарактеристики')
                product_set_code_copy = product_set_code.copy()
                
            # Оставляем в справочние товара только характеристики для выбранного уровня
            product_set_code['КодХарактеристики'] = product_set_code_copy['КодХарактеристики'].apply(lambda x: x.intersection(set_set))
            
            # Объединить список кодов Сегмента во множество
            dim_segment_filter['длина'] = dim_segment_filter['КодыХарактеристик'].apply(lambda x: len(x))
            
            # Зафиксировать максимальную плотность сегмента на своем уровне
            power_list = sorted(list(set(dim_segment_filter['Power'])), reverse=True)
            if level == 1:
                power_segment = power_list[0]
            elif level == 2:
                power_cluster = power_list[0]
            
            ### ФУНКЦИЯ СЕГМЕНТАЦИИ ################################################################################
            
            # Объявление функции
            def select_segment(df_segment, set_x, power_segment=None, segm_lvl=None, level=level, power_list=power_list):
                # Инициация переменных
                inter=0
                segment = ''
                # Перебор плотностей, начиная с максимальной
                for p in power_list:
                    # Условие для первого уровня. Не требует фильтрации по родительскому столбцу
                    if level==1:
                        # Срез сегментов по плотности
                        df_segment_filter = df_segment[df_segment['Power'] == p]
                        # Перебор множеств характеристик из сегментов
                        for set_i in list(df_segment_filter.iloc[:,1]):
                            # Счет количества пересечений характеристик
                            inter = len(set_i.intersection(set_x))
                            # Если количество пересечений соответствует мощности, выводим название этого сегмента
                            if inter >= p:
                                segment = df_segment_filter[df_segment_filter['КодыХарактеристик'] == set_i]['Сегмент'].values[0]
                                return segment
                            
                    # Условие для следующих уровней. Добавлена фильтрация по родительскому столбцу для сохранения иерархии сегментов       
                    elif level>1:
                        # Если родительский столбец является "прочим", подчиненные сегменты тоже "прочие"
                        if segm_lvl == 'Прочее' or len(set_x) <= power_segment:
                            return 'Прочее'
                        else:
                            df_segment_filter = df_segment[(df_segment['Power'] == p) & (df_segment['Родитель'] == segm_lvl) ]
                            for set_i in list(df_segment_filter.iloc[:,1]):
                                inter = len(set_i.intersection(set_x))
                                if inter >= p:
                                    segment = df_segment_filter[df_segment_filter['КодыХарактеристик'] == set_i]['Сегмент'].values[0]
                                    return segment        
                # При невыполнении всех условий сегмент назначается "Прочим"      
                return 'Прочее'
    #############################################################################################################
    
        
        # Имя для столбца в соответствии с текущим уровнем        
        lvl_name = f'Уровень_{level}'
        lvl_name
    
        # Параметры для функции сегментации. На втором и третьем уровне учитывается родительский столбец для поддержания иерархии
        if level==1:
            product_set_code[lvl_name] = product_set_code['КодХарактеристики'].apply(lambda x: select_segment(dim_segment_filter, x, None))
        if level==2:
            product_set_code[lvl_name] = product_set_code.apply(lambda row: select_segment(
                dim_segment_filter, row['КодХарактеристики'], power_segment, row['Уровень_1']
                ), axis=1)
        if level==3:
            product_set_code[lvl_name] = product_set_code.apply(lambda row: select_segment(
                dim_segment_filter, row['КодХарактеристики'], power_cluster, row['Уровень_2']
                ), axis=1) 
        
    
    
    # Переименование прочих сегментов

    product_set_code['Уровень_1'] = product_set_code.apply(lambda row: Category_name + ' // ' + 'Прочее' if row['Уровень_1']=='Прочее' else row['Уровень_1'], axis=1)
    product_set_code['Уровень_2'] = product_set_code.apply(lambda row: row['Уровень_1'] + ' // ' + 'Прочее' \
        if (row['Уровень_2']=='Прочее') or (row['Уровень_2']==row['Уровень_1'])  else row['Уровень_2'], axis=1)
    product_set_code['Уровень_3'] = product_set_code.apply(lambda row: row['Уровень_2'] + ' // ' + 'Прочее' \
        if (row['Уровень_3']=='Прочее') or (row['Уровень_3']==row['Уровень_2'])  else row['Уровень_3'], axis=1)
    product_set_code['КодКатегории'] = Category_code
    
    # Запись выходного файла (категория после 3 циклов обработки)
    
    # with pd.ExcelWriter(f"//Adm-logist-95/Для_регистра_товар_сегмент/{Category_code}.xlsx") as writer:
    #     product_set_code.to_excel(writer, sheet_name="ДляИмпорта", index=False)
    
    print(f'{counter}/{len(list_auto_segments)}) Категория {Category_name} записана')
    
    # Подготовка к записи категории для справочника сегментов
    df_tovar_segment = product_set_code.drop(columns='КодХарактеристики')
    
    lvl_1 = df_tovar_segment[['Уровень_1', 'КодКатегории']].drop_duplicates().rename(columns={'Уровень_1':'Сегмент'})
    lvl_1['Уровень'] = 1
    lvl_2 = df_tovar_segment[['Уровень_2', 'КодКатегории']].drop_duplicates().rename(columns={'Уровень_2':'Сегмент'})
    lvl_2['Уровень'] = 2
    lvl_3 = df_tovar_segment[['Уровень_3', 'КодКатегории']].drop_duplicates().rename(columns={'Уровень_3':'Сегмент'})
    lvl_3['Уровень'] = 3
    category_for_dim_segment = pd.concat([lvl_1, lvl_2, lvl_3], ignore_index=True)
    
    category_for_dim_segment = pd.concat([category_for_dim_segment, df_for_dim_segment_0], ignore_index=True)
    df_for_dim_segment_0 = category_for_dim_segment
    
    # Запись категории для справочника сегментов
    # with pd.ExcelWriter(f"data/Для_Справочника_Сегментов/{Category_code}.xlsx") as writer:
    #     category_for_dim_segment.to_excel(writer, sheet_name="ДляИмпорта", index=False)
    
    df_tovar_segment = pd.concat([df_tovar_segment_0, df_tovar_segment], ignore_index=True)
    df_tovar_segment_0 = df_tovar_segment
    
# Запись выходного файла по всем категориям
# with pd.ExcelWriter("data/"+str(datetime.datetime.now().strftime("%m-%d-%Y_%H-%M"))+".xlsx") as writer:
#     df_tovar_segment.to_excel(writer, sheet_name="ДляИмпорта", index=False)

print()
print('************************************************')
print('Таблица Товар-Сегмент')
print()
df_tovar_segment.info()
print()
display(df_tovar_segment.head())
print()
print('Таблица Справочник Сегментов')
print()
category_for_dim_segment.info()
print()
display(category_for_dim_segment.head())


Обработка категории : AM18166 Блок питания системного блока


1/15) Категория Блок питания системного блока записана

Обработка категории : AM18156 GPS Навигатор
Уровень 3 отсутствует в справочнике сегментов
2/15) Категория GPS Навигатор записана

Обработка категории : AM18165 Корпус
3/15) Категория Корпус записана

Обработка категории : AM18166 Блок питания системного блока
4/15) Категория Блок питания системного блока записана

Обработка категории : AM18168 Твердотельный накопитель SSD
5/15) Категория Твердотельный накопитель SSD записана

Обработка категории : AM18171 Оперативная память
6/15) Категория Оперативная память записана

Обработка категории : AM18172 Материнская плата
7/15) Категория Материнская плата записана

Обработка категории : AM18173 Видеокарта
8/15) Категория Видеокарта записана

Обработка категории : AM18176 Моноблок
9/15) Категория Моноблок записана

Обработка категории : AM18178 Персональный компьютер офисный
10/15) Категория Персональный компьютер офисный записана

Обработка категории : AM18179 Микрокомпьютеры, стики, пла

,Код,Уровень_1,Уровень_2,Уровень_3,КодКатегории
0,1000717,Блок питания системного блока // Прочее,Блок питания системного блока // Прочее // Прочее,Блок питания системного блока // Прочее // Про...,AM18166
1,1001923,Блок питания системного блока // Прочее,Блок питания системного блока // Прочее // Прочее,Блок питания системного блока // Прочее // Про...,AM18166
2,1001927,Блок питания системного блока // Прочее,Блок питания системного блока // Прочее // Прочее,Блок питания системного блока // Прочее // Про...,AM18166
3,1001928,Блок питания системного блока // Прочее,Блок питания системного блока // Прочее // Прочее,Блок питания системного блока // Прочее // Про...,AM18166
4,1004162,Мощность (номинал)_4) 800.0-1250.0 // Отстегив...,Мощность (номинал)_4) 800.0-1250.0 // Отстегив...,Мощность (номинал)_4) 800.0-1250.0 // Отстегив...,AM18166



Таблица Справочник Сегментов

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1399 entries, 0 to 1398
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Сегмент       1399 non-null   object
 1   КодКатегории  1399 non-null   object
 2   Уровень       1399 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 32.9+ KB



,Сегмент,КодКатегории,Уровень
0,"Сумки, чехлы для ноутбуков // Прочее",AM18200,1
1,Основной цвет_черный // Материал_нейлон // Вне...,AM18200,1
2,Основной цвет_черный,AM18200,1
3,Основной цвет_белый,AM18200,1
4,Основной цвет_серый,AM18200,1


# 4. Итоги

Итогом проделанной работы стали две таблицы, которые можно использовать в качестве справочника сегментов и регистра связей товара с сегментами всех уровней.
Сегментация обладает следующими свойствами:

    1.	Нет повторяющихся наименований сегментов.
Если сегмент для нового товара не определяется (то есть является «Прочим»), то ему присваивается название из имени категории + Прочее

    2.	Все сегменты имеют понятную структуру, названия логически интерпретируются.
Например, если для товара находится подходящая ячейка, она «прилипает» к товару. Если подходящей ячейки не находится, то наследуется имя предыдущего уровня (кластера) с припиской «Прочее».
На уровне Кластер-Сегмент – то же самое, если кластера нет, то наследуется имя сегмента с припиской «Прочее».
И если какой-то из уровней становится «прочим», все его подчиненные также будут «прочими». 

    3.	За счет четкой иерархии есть возможность логически работать с любым уровнем сегментации.
Имеется в виду, что даже работая на уровне ячейки, уже из названия понятна ее «плотность», и в каком она кластере и сегменте.


### Проверка полноты выполнения задач и требований к результату.

**Технические задачи** 

1. Разработать алгоритм отбора наиболее подходящих технических характеристик для сегментации - Выполнено.
2. Произвести сегментацию по отобранным характеристикам - Выполнено.
3. Выгрузить модель данных в Excel для возможности оценки модели заказчиком - Выполнено.
4. Подготовить таблицу - справочник сегментов (для дальнейшей интеграции в 1С) - Выполнено.
5. Настроить процесс классификации нового товара по сегментам из справочника - Выполнено.
6. Подготовить таблицу с результатами классификации - Выполнено.

**Требования к результату** 

    1. Модель должна быть легко интерпретируема, любой пользователь должен понимать, почему этот товар находится именно в этом сегменте.
Поскольку названия сегментов состоят из названий технических характеристик, любому пользователю понятно, для какого товара сделан сегмент.

    2. Сегмент первого уровня не должен содержать менее 10 номенклатурных позиций.
Данное условие заложено в настройках ограничений. Сегменты первого уровня содержат минимум 10-30 номенклатурных позиций.

    3. Справочник сегментов должен строиться только на основании товара с остатками.
В качестве исходных данных для модели использовались только номенклатурные позиции с остатками.

    4. Классификация по сегментам должна происходить по всей товарной номенклатуре. Если появляется новый товар, для него должны подбираться оптимальные сегменты.
На вход алгоритма классификации подается вся номенклатура, включая товар без остатка. И для каждого товара присваивается свой сегмент.